<a href="https://colab.research.google.com/github/Praneesh-Sharma/Deep-Learning-Practice/blob/main/NLP/Word_Embeddings_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word Embeddings for Sentiment Analysis

This notebook explains how to add the embeddings layer to the neural network. We will train our own word embeddings using a simple Keras model for a sentiment classification task.

Steps include:
1. Downloading data from tensorflow dataset.
2. Segregating training and testing sentences & labels.
3. Data preparation to padded sequences
4. Defining out Keras model with an Embedding layer.
5. Train the model and explore the weights from the embedding layer.

In [2]:
##import the required libraries and APIs
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.12.0


## Downloading the TensorFlow `imdb_review` dataset

> Make sure tensorflow_datasets is installed

In [3]:
##load the imdb reviews dataset
data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWWVR6F/imdb_reviews-train.tfrecord…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWWVR6F/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWWVR6F/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Segregating training and testing sets

In [4]:
##segregate training and test set
train_data, test_data = data['train'], data['test']

##create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [5]:
##iterate over the train data to extract sentences and labels
for sent, label in train_data:
    train_sentences.append(str(sent.numpy().decode('utf8')))
    train_labels.append(label.numpy())

##iterate over the test set to extract sentences and labels
for sent, label in test_data:
    test_sentences.append(str(sent.numpy().decode('utf8')))
    test_labels.append(label.numpy())



In [6]:
##convert lists into numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Data preparation - setting up the tokenizer

In [7]:
##define the parameters for the tokenizing and padding
vocab_size = 10000
embedding_dim = 16
max_length = 150
trunc_type='post'
oov_tok = "<oov>"


In [8]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

##training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen=max_length, truncating=trunc_type)

##testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length)

In [9]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   11   26   75  571
    6  805 2354  313  106   19   12    7  629  686    6    4 2219    5
  181  584   64 1454  110 2263    3 3951   21    2    1    3  258   41
 4677    4  174  188   21  

## Define the Neural Network with Embedding layer

1. Use the Sequential API.
2. Add an embedding input layer of input size equal to vocabulary size.
3. Add a flatten layer, and two dense layers.

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

##compile the model with loss function, optimizer and metrics
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 2400)              0         
                                                                 
 dense (Dense)               (None, 6)                 14406     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 174,413
Trainable params: 174,413
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)    

## Model Training

In [12]:
num_epochs = 10

##train the model with training and validation set
model.fit(
    train_padded, 
    train_labels, 
    epochs=num_epochs, 
    validation_data=(test_padded, test_labels)
    )

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.4904 - accuracy: 0.7390 - val_loss: 0.3422 - val_accuracy: 0.8519
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 0.2239 - accuracy: 0.9154 - val_loss: 0.3399 - val_accuracy: 0.8550
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0862 - accuracy: 0.9785 - val_loss: 0.4059 - val_accuracy: 0.8478
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0241 - accuracy: 0.9965 - val_loss: 0.4858 - val_accuracy: 0.8408
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0064 - accuracy: 0.9996 - val_loss: 0.5353 - val_accuracy: 0.8451
Epoch 6/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.5807 - val_accuracy: 0.8464
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 8.7415e-04 - accuracy: 1.0000 - val_loss: 0.6193 - val_accuracy: 0.8459


## Deriving weights from the embedding layer

In [13]:
##isolating the first embedding layer
l1 = model.layers[0]

##extracting learned weights
weights = l1.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
print(weights)

(10000, 16)
[[-0.02061544 -0.01469837  0.01530938 ... -0.01285964 -0.00394849
   0.01510541]
 [ 0.02406914 -0.03008548  0.07439308 ...  0.02155243 -0.02213802
   0.05547312]
 [ 0.05141378 -0.00956256  0.08798461 ...  0.04572194 -0.03780771
   0.0503942 ]
 ...
 [ 0.06829984  0.1445302  -0.04647512 ... -0.15313162  0.17314498
   0.07238512]
 [-0.05198441  0.13767493  0.02666548 ... -0.09318875  0.08404788
  -0.02727952]
 [ 0.12107612  0.04673135  0.0598953  ... -0.09453202  0.06615947
   0.06149711]]
